In [2]:
using CSV, DataFrames
using FileIO, Dates
using Graphs

include("../region_cube_split/cubes.jl")

region_cube_split (generic function with 1 method)

In [3]:
romania_full = CSV.read("../data/romania.csv", DataFrame);
romania = romania_full[romania_full.Datetime .> DateTime(1976,1,1,0,0,0),:];

vrancea = romania[(romania.Latitude .>= 45.20) .& (romania.Latitude .<= 46.0) .& 
                    (romania.Longitude .>= 26.0) .& (romania.Longitude .<= 27.0) .&
                    (romania.Depth .>= 50.0) .& (romania.Depth .<= 200.0), :];

In [4]:
vrancea, vrancea_cubes = region_cube_split(vrancea,side=5);

In [5]:
cube_dictionary = Dict()
for i=1:length(vrancea_cubes.cubeIndex)
    cube_dictionary[vrancea_cubes.cubeIndex[i]] = i
end
cube_dictionary;

In [6]:
G = Graphs.Graph(length(vrancea_cubes.cubeIndex))

{1727, 0} undirected simple Int64 graph

In [7]:
edgeWeight=false

false

In [8]:
i=0
while i < length(vrancea.cubeIndex)-1

    i+=1
    
    current_node = cube_dictionary[vrancea.cubeIndex[i]]
    target_node = cube_dictionary[vrancea.cubeIndex[i+1]]

    if edgeWeight == true
        add_edge!(G,current_node,target_node)

    else
        if has_edge(G, current_node, target_node)
            continue

        else
            add_edge!(G,current_node,target_node)
        end

    end
    
    # Break from loop because I am only interested in the next quake chronologically

    

end

In [9]:
G

{1727, 7612} undirected simple Int64 graph

In [34]:
# For edge weights must use:
using SimpleWeightedGraphs


# USE METAGRAPHS FOR PROPERTIES ON VERTICES

In [35]:
G = SimpleWeightedGraph(3)

{3, 0} undirected simple Int64 graph with Float64 weights

In [37]:
add_edge!(G,1,2)

true

In [38]:
G

{3, 1} undirected simple Int64 graph with Float64 weights

In [39]:
add_edge!(G,1,2)

true

In [40]:
G

{3, 1} undirected simple Int64 graph with Float64 weights

In [52]:
G.weights[1,2]

2.0

In [56]:
G.weights[2,1]+=1

2.0

In [54]:
get_weight(G,2,1)

2.0

In [57]:
get_weight(G,1,2)

2.0

In [18]:
G = Graphs.Graph(3)

{3, 0} undirected simple Int64 graph

In [30]:
G = Graphs.SimpleGraphs.AbstractSimpleGraph

Graphs.SimpleGraphs.AbstractSimpleGraph

In [19]:
add_edge!(G,1,2)

true

In [31]:
G

Graphs.SimpleGraphs.AbstractSimpleGraph